# การพัฒนา Joy Compiler (เฟส 3)

ในเฟสที่สามนี้ เราจะพัฒนา Joy 2.0 โดยต่อยอดจาก Joy 1.0 เพื่อให้มีฟีเจอร์เพิ่มเติมดังนี้
* รองรับคอมเม้นต์แบบบรรทัดเดียว
* รองรับ literal integer ในรูปเลขฐานสอง (เช่น `0b101101`) และฐานสิบหก (เช่น `0x3f58`)
* รองรับ character literal เช่น `'a'`, `'x'`
* รองรับการอ้างถึงหน่วยความจำในรูปพอยเตอร์ เพื่อนำไปใช้งานกับสตริง อาร์เรย์ และอินพุทเอาท์พุทในรูปแบบ memory-mapped I/O
* รองรับการสร้างสตริง และอาร์เรย์


# ดึงโค้ดจาก Joy 1.0

เหมือนที่ผ่านมา เราจะดึงโค้ดจาก Joy 1.0 เพื่อมาต่อยอดเป็น Joy 2.0 ให้ป้อน file id ของ colab notebook ที่ใช้ในงาน Joy 1.0 (Joy compiler - phase 2) ด้านล่าง และรันเซลล์ที่เหลือเพื่อดาวน์โหลดและอิมพอร์ต Joy 1.0 มาใช้ในโน้ตบุ๊คนี้

In [ ]:
JOY10_COLAB_FILE_ID = '1k_w3wTUyid-uyH4UbeNJYUfDyrzq6wgm'

In [ ]:
%%capture
!pip install PyDrive
!wget -q https://ecourse.cpe.ku.ac.th/courses/comsys/lib/comsys_import_ipynb.py

In [ ]:
from pydrive2.auth import GoogleAuth
from pydrive2.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

downloaded = drive.CreateFile({'id': JOY10_COLAB_FILE_ID})
downloaded.GetContentFile('joy10.ipynb')

ขั้นตอนด้านล่างอาจใช้เวลาค่อนข้างนาน เนื่องจากการอิมพอร์ท joy10 ทำให้เกิดการอิมพอร์ท joy05 ที่ทำมาก่อนหน้านั้นด้วย

In [ ]:
%%capture
import comsys_import_ipynb
from joy10 import (
    draw, listing, assemble, simulate, assemble_and_simulate,
    Joy07, Joy10, TestJoy07, TestJoy10, run_test)

รันตัวทดสอบโค้ดของ Joy 1.0 เพื่อให้แน่ใจว่าอิมพอร์ตโค้ดเดิมมาอย่างถูกต้อง

In [ ]:
run_test(TestJoy10)

..........................
----------------------------------------------------------------------
Ran 26 tests in 1.299s

OK


# Joy 1.1 - comments
ภาษา Joy 1.1 รองรับทุกฟีเจอร์ของ Joy 1.0 โดยเพิ่มฟีเจอร์ดังนี้

---
* รองรับโค้ดคอมเม้นต์แบบบรรทัดเดียวในรูป `//`
---

โค้ดคอมเม้นต์ถูกประมวลผลตั้งแต่ขั้นตอน lexer ดังนั้นให้นิยามเทอร์มินัล COMMENT และสั่ง ignore โดยคัดลอกแกรมมาร์จากคลาส Joy10 มาวางไว้ในคลาส Joy11 ด้านล่าง แล้วเพิ่มสองบรรทัดต่อไปนี้ลงไป

```
    COMMENT: "//" /[^\r\n]*[\r\n]?/
    %ignore COMMENT
```

เนื่องจากโทเค็นคอมเม้นต์ไม่ถูกส่งต่อมายังพาร์เซอร์ จึงไม่จำเป็นต้องแก้ไขส่วนของโค้ดเจเนอเรชันใด ๆ

In [ ]:
# DO NOT ERASE THIS CELL - to be graded

class Joy11(Joy10):
    GRAMMAR = r'''
        // put Joy11's grammar here
        program: statements
        statements: statement*
        statement: stmt_assign
                 | stmt_if
                 | stmt_if_else
                 | stmt_while
                 | stmt_until
        stmt_assign: "let" ID "=" expr ";"
        stmt_if: "if" expr "{" statements "}"
        stmt_if_else: "if" expr "{" statements "}" "else" "{" statements "}"
        stmt_while: "while" expr "{" statements "}"
        stmt_until: "until" expr "{" statements "}"
        expr: expr_const
            | expr_id
            | expr_add
            | expr_b_sub
            | expr_u_sub
            | expr_bracket
            | expr_comp
            | expr_logic
        expr_const: NUMBER
        expr_id: ID
        expr_add: expr "+" expr
        expr_b_sub: expr "-" expr
        expr_u_sub: "-" expr
        expr_bracket: "(" expr ")"
        expr_comp: expr COMPARE_OP expr
        expr_logic: expr LOGIC_BIN_OP expr
                  | LOGIC_UN_OP expr

        ID: /[_A-Za-z][_0-9A-Za-z]*/
        NUMBER: /-?[0-9]+/
        COMPARE_OP: ">=" | "<=" | "==" | ">" | "<" | "!="
        LOGIC_BIN_OP: "&&" | "||"
        LOGIC_UN_OP: "!"
        WS: /[ \t\f\r\n]+/
        COMMENT: "//" /[^\r\n]*[\r\n]?/
        %ignore WS
        %ignore COMMENT
    '''

## Test case

In [ ]:
class TestJoy11(TestJoy10):

    JOY_COMPILER = Joy11

    def test_comment(self):
        self.compile_and_run('''
            // let a = 8;
            let b = 3;
            let c = 1234; // let b = 10;
        ''')
        self.assertEqual(self.cpu.ram[16], 3)
        self.assertEqual(self.cpu.ram[17], 1234)

run_test(TestJoy11)

...........................
----------------------------------------------------------------------
Ran 27 tests in 1.469s

OK


# Joy 1.2 - integer/char literals
Joy 1.2 รองรับฟีเจอร์ทั้งหมดของ Joy 1.1 และเพิ่มฟีเจอร์ดังนี้

---
* รองรับ integer literal <u>ฐานสิบหก</u> ด้วยพรีฟิกซ์ `0x` เช่น `0x1ac2` ให้ค่า 6850
* รองรับ integer literal <u>ฐานสอง</u> ด้วยพรีฟิกซ์ `0b` เช่น `0b1011010010` ให้ค่า 722
* รองรับ character literal ในรูป `'x'` ซึ่งจะให้ค่าเป็นจำนวนเต็มแทนรหัสแอสกี้ของอักขระในเครื่องหมาย `''` เช่น `'A'` ให้ค่า 65
---

<u>แนวคิด</u>
* แก้ไขเมท็อด `expr_const` ทับตัวที่อยู่ในคลาสเดิม
* ไพทอนมีฟังก์ชัน `int` ที่ใช้สำหรับแปลงค่าตัวเลขจากฐานที่ต้องการพร้อมให้ใช้งานอยู่แล้ว​ (ลองศึกษาการเรียกใช้ฟังก์ชัน `int` ด้วยอาร์กิวเมนต์ `base=0` ดู)

In [ ]:
# DO NOT ERASE THIS CELL - to be graded

class Joy12(Joy11):
    GRAMMAR = r'''
        // put Joy12's grammar here
        program: statements
        statements: statement*
        statement: stmt_assign
                 | stmt_if
                 | stmt_if_else
                 | stmt_while
                 | stmt_until
        stmt_assign: "let" ID "=" expr ";"
        stmt_if: "if" expr "{" statements "}"
        stmt_if_else: "if" expr "{" statements "}" "else" "{" statements "}"
        stmt_while: "while" expr "{" statements "}"
        stmt_until: "until" expr "{" statements "}"
        expr: expr_const
            | expr_id
            | expr_add
            | expr_b_sub
            | expr_u_sub
            | expr_bracket
            | expr_comp
            | expr_logic
        expr_const: NUMBER
        expr_id: ID
        expr_add: expr "+" expr
        expr_b_sub: expr "-" expr
        expr_u_sub: "-" expr
        expr_bracket: "(" expr ")"
        expr_comp: expr COMPARE_OP expr
        expr_logic: expr LOGIC_BIN_OP expr
                  | LOGIC_UN_OP expr

        ID: /[_A-Za-z][_0-9A-Za-z]*/
        NUMBER: /-?[0-9]+/ | "0x"/[0-9A-Fa-f]+/ | "0b"/[01]+/ | /'[^']'/
        COMPARE_OP: ">=" | "<=" | "==" | ">" | "<" | "!="
        LOGIC_BIN_OP: "&&" | "||"
        LOGIC_UN_OP: "!"
        WS: /[ \t\f\r\n]+/
        COMMENT: "//" /[^\r\n]*[\r\n]?/
        %ignore WS
        %ignore COMMENT
    '''

    # override Joy11's expr_const
    def expr_const(self, tree):
        # put your code here
        [const] = tree.children
        text = str(const)
        if text.startswith("'") and text.endswith("'") and len(text) == 3:
           value = ord(text[1])
        else:
           value = int(text,0)
        self.gen_load_const('THIS', value)

## Test case

In [ ]:
# DO NOT MODIFY

class TestJoy12(TestJoy11):

    JOY_COMPILER = Joy12

    def test_const_hex(self):
        self.compile_and_run('let a = 0x1234;')
        self.assertEqual(self.cpu.ram[16], 0x1234)

    def test_const_bin(self):
        self.compile_and_run('let a = 0b0101010101010101;')
        self.assertEqual(self.cpu.ram[16], 0b0101010101010101)
        self.compile_and_run('let a = 0b1010101010101010;')
        self.assertEqual(self.cpu.ram[16], 0b1010101010101010)

    def _do_test_const_char(self, char):
        self.compile_and_run(f'''
            let a = '{char}';
        ''')
        self.assertEqual(self.cpu.ram[16], ord(char))

    def test_const_character(self):
        for i in range(32, 127):
            c = chr(i)
            if c == "'":
                continue
            self._do_test_const_char(c)

run_test(TestJoy12)

..............................
----------------------------------------------------------------------
Ran 30 tests in 3.831s

OK


# Joy 1.3 - pointers
Joy 1.3 รองรับฟีเจอร์ทั้งหมดของ Joy 1.2 พร้อมฟีเจอร์เพิ่มเติมดังนี้

---
* รองรับการเข้าถึงหน่วยความจำแบบพอยเตอร์ ผ่านตัวดำเนินการ `&` (reference) และ `*` (dereference)
    * นิพจน์ &*ID* ให้ค่าเป็นตำแหน่งหน่วยความจำที่เก็บค่าตัวแปร *ID*
    * นิพจน์ **expr* อ้างถึงตำแหน่งหน่วยความจำในตำแหน่งที่ตรงกับค่าของ *expr*
---

ตัวอย่างเช่น
```
let a = 8;   // กำหนดให้ a เท่ากับ 8
let b = &a;  // กำหนดให้ b เท่ากับตำแหน่งหน่วยความจำของ a
let c = *b;  // กำหนดให้ c เท่ากับค่าในหน่วยความจำที่ b ชี้อยู่ ซึ่งเท่ากับ 8 (ค่า a)
let *b = 30; // กำหนดให้ตำแหน่งหน่วยความจำที่ b ชี้อยู่มีค่า 30 จึงมีผลทำให้ a เท่ากับ 30
```

<u>คำใบ้:</u> แกรมมาร์ไม่ได้ปรับเพิ่มเพียงแค่ส่วนของ *expr* แต่ต้องปรับเพิ่มกฎของคำสั่ง `let` ด้วยเช่นกัน เพื่อให้รองรับคำสั่งในรูปแบบตามตัวอย่างสุดท้าย

In [ ]:
# DO NOT ERASE THIS CELL - to be graded

class Joy13(Joy12):
    GRAMMAR = r'''
        //Put Joy13's grammar here
        program: statements
        statements: statement*
        statement: stmt_assign
                 | stmt_if
                 | stmt_if_else
                 | stmt_while
                 | stmt_until
                 | stmt_assign_deref

        stmt_assign: "let" ID "=" expr ";"
        stmt_if: "if" expr "{" statements "}"
        stmt_if_else: "if" expr "{" statements "}" "else" "{" statements "}"
        stmt_while: "while" expr "{" statements "}"
        stmt_until: "until" expr "{" statements "}"
        stmt_assign_deref: "let" "*" expr "=" expr ";"

        expr: expr_const
            | expr_id
            | expr_add
            | expr_b_sub
            | expr_u_sub
            | expr_bracket
            | expr_comp
            | expr_logic
            | expr_ref
            | expr_deref
        expr_const: NUMBER
        expr_id: ID
        expr_add: expr "+" expr
        expr_b_sub: expr "-" expr
        expr_u_sub: "-" expr
        expr_bracket: "(" expr ")"
        expr_comp: expr COMPARE_OP expr
        expr_logic: expr LOGIC_BIN_OP expr
                  | LOGIC_UN_OP expr
        expr_ref: "&" ID
        expr_deref: "*" expr

        ID: /[_A-Za-z][_0-9A-Za-z]*/
        NUMBER: /-?[0-9]+/ | "0x"/[0-9A-Fa-f]+/ | "0b"/[01]+/ | /'[^']'/
        COMPARE_OP: ">=" | "<=" | "==" | ">" | "<" | "!="
        LOGIC_BIN_OP: "&&" | "||"
        LOGIC_UN_OP: "!"
        WS: /[ \t\f\r\n]+/
        COMMENT: "//" /[^\r\n]*[\r\n]?/
        %ignore WS
        %ignore COMMENT
    '''

    # define your code-generation methods here
    def expr_ref(self,tree):
        [id] = tree.children
        self._asm.append(f'''
            // [THIS] <- address of @.var.{id}
            @.var.{id}
            D=A
            @THIS
            M=D
        ''')

    def expr_deref(self,tree):
        [expr] = tree.children
        self.visit(expr)
        self._asm.append(f'''
           // [THIS] <- Value in address of [THIS]
           @THIS
           A=M
           D=M
           @THIS
           M=D
        ''')
    def stmt_assign_deref(self,tree):
        [left_expr,right_expr] = tree.children
        self.visit(left_expr)
        self._asm.append(f'''
            //Save value in R5
            @THIS
            D=M
            @R5
            M=D
        ''')
        self.visit(right_expr)
        self._asm.append(f'''
            //Store from R5
            @THIS
            D=M
            @R5
            A=M
            M=D
        ''')

## Test case

In [ ]:
# DO NOT MODIFY

class TestJoy13(TestJoy12):

    JOY_COMPILER = Joy13

    def test_expr_ref(self):
        self.compile_and_run('''
            let a = 8;
            let b = &a;
        ''')
        self.assertEqual(self.cpu.ram[16], 8)
        self.assertEqual(self.cpu.ram[17], 16)
        self.assertEqual(self.cpu.ram[0], 0x3fff)

    def test_expr_deref(self):
        self.compile_and_run('''
            let a = 8;
            let b = &a;
            let a = 29;
            let c = (*b) + 10;
        ''')
        self.assertEqual(self.cpu.ram[16], 29)
        self.assertEqual(self.cpu.ram[17], 16)
        self.assertEqual(self.cpu.ram[18], 29+10)
        self.assertEqual(self.cpu.ram[0], 0x3fff)

    def test_stmt_assign_deref(self):
        self.compile_and_run('''
            let a = 8;
            let *a = 1234;
            let *(a+1) = 4321;
        ''')
        self.assertEqual(self.cpu.ram[8], 1234)
        self.assertEqual(self.cpu.ram[9], 4321)
        self.assertEqual(self.cpu.ram[0], 0x3fff)

    def test_deref_arithmetic(self):
        self.compile_and_run('''
            let *10 = 1;
            let *11 = 2;
            let *12 = 3;
            let a = 10;
            let b = *a;     // 1
            let c = *(a+1); // 2
            let d = *(a+2); // 3
            let *13 = c;
            let *14 = d;
        ''')
        self.assertEqual(self.cpu.ram[13], 2)
        self.assertEqual(self.cpu.ram[14], 3)

    def test_stmt_deref_arithmetic(self):
        self.compile_and_run('''
            let a = 10;
            let b = 2;
            let *a = 5;       // *10 = 5
            let *(a+1) = 6;   // *11 = 6
            let *(a+b) = 7;   // *12 = 7
            let *(a+3) = b+8; // *13 = 10
        ''')
        self.assertEqual(self.cpu.ram[10], 5)
        self.assertEqual(self.cpu.ram[11], 6)
        self.assertEqual(self.cpu.ram[12], 7)
        self.assertEqual(self.cpu.ram[13], 10)

run_test(TestJoy13)

...................................
----------------------------------------------------------------------
Ran 35 tests in 3.980s

OK


ฟีเจอร์ของ Joy 1.3 ทำให้เราสามารถอ้างถึงหน่วยความจำที่ตำแหน่งใดก็ได้ รวมถึงหน่วยความจำที่เข้าถึง Screen (ตำแหน่งที่ 0x4000 ถึง 0x5FFF) และ Keyboard (ตำแหน่งที่ 0x6000)

โค้ดด้านล่างสาธิตความสามารถของ Joy 1.3 ในการเขียนโปรแกรมให้มีพฤติกรรมเช่นเดียวกับโปรแกรม `Fill.asm` ที่เขียนด้วยภาษาแอสเซมบลี้ก่อนหน้านี้ โดยระบายสีหน้าจอให้เป็นสีดำเมื่อมีการกดคีย์ใด ๆ และระบายสีขาวเมื่อปล่อยคีย์

In [ ]:
asm = Joy13().compile('''
    while 1 {
        let count = 0;
        until count >= 0x2000 {
            if *0x6000 {
                let *(0x4000+count) = 0xffff;
            } else {
                let *(0x4000+count) = 0x0000;
            }
            let count = count + 1;
        }
    }
''')
assemble_and_simulate(asm, super_fast=True)

# Joy 2.0 - strings and arrays
Joy 2.0 รองรับทุกฟีเจอร์ของ Joy 1.3 และเพิ่มฟีเจอร์ต่อไปนี้

* รองรับนิพจน์แบบสตริง (string) ในรูป `"xxxx"` นิพจน์นี้จะคืนค่าตำแหน่งหน่วยความจำที่เก็บอักขระแรกของสตริง โดยอักขระที่เหลือจะต้องอยู่ในหน่วยความจำที่ติดกันตำแหน่งต่อไปเรื่อย ๆ (หนึ่งอักขระใช้พื้นที่หนึ่งเวิร์ด) และปิดท้ายด้วยค่า 0 เช่นเดียวกับสตริงในภาษาซี
* รองรับนิพจน์แบบอาร์เรย์ (array) ในรูป `[`*expr1*`,`*expr2*`,`...`]` ซึ่งจะให้ค่าเป็นตำแหน่งหน่วยความจำที่เก็บค่า *expr1* และหน่วยความจำติดกันตำแหน่งถัดไปจะเก็บค่า *expr2*, *expr3*, ... ไปเรื่อย ๆ แต่ไม่ต้องปิดท้ายด้วย 0 เหมือนสตริง

ตัวอย่างเช่น
```
let a = "joy";
let b = [12, 3+4, 'A', 0x10];
```

จะต้องมีผลทำให้มีค่ารหัสแอสกี้ของ `'j'`, `'o'`, `'y'` และศูนย์ เรียงกันอยู่ในที่ใดที่หนึ่งในแรม โดยตัวแปร `a` มีค่าเท่ากับตำแหน่งหน่วยความจำที่เก็บ `'j'` และจะต้องมีค่า 12, 7, 65 และ 16 เรียงกันอยู่ที่ใดที่หนึ่งในแรม โดยตัวแปร `b` มีค่าเท่ากับตำแหน่งหน่วยความจำที่เก็บค่า 12 ดังภาพ

```
          RAM
       +-------+
       |   :   |
       +-------+
     a |  ???  | ----+
       +-------+     |
       |   :   |     |
       +-------+     |
     b |  ???  | ----------+
       +-------+     |     |
       |       |     |     |
       |       |     |     |
       |       |     |     |      addr ต่ำ
       |       |     |     |        |
       +-------+     |     |        |
       |  'j'  | <---+     |        V
       +-------+           |      addr สูง
       |  'o'  |           |
       +-------+           |
       |  'y'  |           |
       +-------+           |
       |   0   |           |
       +-------+           |
       |   :   |           |
       +-------+           |
       |  12   | <---------+
       +-------+
       |   7   |
       +-------+
       |  65   |
       +-------+
       |  16   |
       +-------+
       |       |
       +-------+
```

<u>แนวคิด</u>
* สตริงเป็นโทเค็นชนิดใหม่ อย่าลืมนิยามเทอร์มินัลใหม่เพิ่มเติม​
* ไม่ต้องพะวงกับประสิทธิภาพของโค้ดแอสเซมบลี้มากนัก วิธีที่ง่ายที่สุดคืออาศัยความสามารถของแอสเซมเบลอร์ในการจองหน่วยความจำให้ตัวแปรโดยสร้างรายการคำสั่งที่ทำให้เกิดการจองหน่วยความจำให้เรียงติดกันเอาไว้ก่อนล่วงหน้า เช่น `@.array.5.0` `@.array.5.1` ...  แล้วค่อยสร้างโค้ดนำค่าที่ถูกต้องมาใส่ลงไปทีละตัว ซึ่งเทคนิคนี้นำไปใช้ได้ทั้งกับอาร์เรย์และสตริง
* จากภาพตัวอย่างข้างต้น ตัวแปร `a` และ `b` อาจปรากฏในหน่วยความจำก่อนหรือหลังอาร์เรย์/สตริงก็ได้ ไม่จำเป็นต้องปรากฏก่อน
* แก้ไขเมท็อด `__init__()` ตามความเหมาะสม หากต้องการสร้างตัวแปรที่เก็บหมายเลขอาร์เรย์/สตริง

In [ ]:
# DO NOT ERASE THIS CELL - to be graded

class Joy20(Joy13):
    GRAMMAR = r'''
        // put Joy20's grammar here
        program: statements
        statements: statement*
        statement: stmt_assign
                 | stmt_if
                 | stmt_if_else
                 | stmt_while
                 | stmt_until
                 | stmt_assign_deref

        stmt_assign: "let" ID "=" expr ";"
        stmt_if: "if" expr "{" statements "}"
        stmt_if_else: "if" expr "{" statements "}" "else" "{" statements "}"
        stmt_while: "while" expr "{" statements "}"
        stmt_until: "until" expr "{" statements "}"
        stmt_assign_deref: "let" "*" expr "=" expr ";"

        expr: expr_const
            | expr_id
            | expr_add
            | expr_b_sub
            | expr_u_sub
            | expr_bracket
            | expr_comp
            | expr_logic
            | expr_ref
            | expr_deref
            | expr_string
            | expr_array
        expr_const: NUMBER
        expr_id: ID
        expr_add: expr "+" expr
        expr_b_sub: expr "-" expr
        expr_u_sub: "-" expr
        expr_bracket: "(" expr ")"
        expr_comp: expr COMPARE_OP expr
        expr_logic: expr LOGIC_BIN_OP expr
                  | LOGIC_UN_OP expr
        expr_ref: "&" ID
        expr_deref: "*" expr
        expr_string: ESCAPED_STRING
        expr_array: "[" [expr ("," expr)*] "]"

        ID: /[_A-Za-z][_0-9A-Za-z]*/
        NUMBER: /-?[0-9]+/ | "0x"/[0-9A-Fa-f]+/ | "0b"/[01]+/ | /'[^']'/
        COMPARE_OP: ">=" | "<=" | "==" | ">" | "<" | "!="
        LOGIC_BIN_OP: "&&" | "||"
        LOGIC_UN_OP: "!"
        WS: /[ \t\f\r\n]+/
        COMMENT: "//" /[^\r\n]*[\r\n]?/
        ESCAPED_STRING: /"([^"\\]|\\.)*"/
        %ignore WS
        %ignore COMMENT
    '''

    # define your code-generation methods, as well as __init__() here
    def __init__(self):
        super().__init__()
        self._heap_counter = 100  # เริ่ม heap ที่ addr=100

    def allocate(self, size):
        addr = self._heap_counter
        self._heap_counter += size
        return addr

    def expr_string(self, tree):
        raw = tree.children[0].value[1:-1]  # ตัด "..."
        chars = [ord(c) for c in raw] + [0]
        addr = self.allocate(len(chars))
        for i, c in enumerate(chars):
            self._asm.append(f'''
                @{c}
                D=A
                @{addr+i}
                M=D
            ''')
        self._asm.append(f'''
            // string literal -> address {addr}
            @{addr}
            D=A
            @THIS
            M=D
        ''')

    def expr_array(self, tree):
        elems = tree.children
        addr = self.allocate(len(elems))
        for i, e in enumerate(elems):
            self.visit(e)
            self._asm.append(f'''
                @THIS
                D=M
                @{addr+i}
                M=D
            ''')
        self._asm.append(f'''
            // array literal -> address {addr}
            @{addr}
            D=A
            @THIS
            M=D
        ''')

## Test case

In [ ]:
# DO NOT MODIFY

class TestJoy20(TestJoy13):

    JOY_COMPILER = Joy20

    def test_string(self):
        self.compile_and_run('''
            let s1 = "Hello";
            let *10 = s1;
            let *11 = "Bye";
        ''')
        addr = self.cpu.ram[10]
        self.assertEqual(self.cpu.ram[addr+0], ord('H'))
        self.assertEqual(self.cpu.ram[addr+1], ord('e'))
        self.assertEqual(self.cpu.ram[addr+2], ord('l'))
        self.assertEqual(self.cpu.ram[addr+3], ord('l'))
        self.assertEqual(self.cpu.ram[addr+4], ord('o'))
        self.assertEqual(self.cpu.ram[addr+5], 0)
        addr = self.cpu.ram[11]
        self.assertEqual(self.cpu.ram[addr+0], ord('B'))
        self.assertEqual(self.cpu.ram[addr+1], ord('y'))
        self.assertEqual(self.cpu.ram[addr+2], ord('e'))
        self.assertEqual(self.cpu.ram[addr+3], 0)
        self.assertEqual(self.cpu.ram[0], 0x3fff)

    def test_string_all_chars(self):
        all_chars = " !#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZ[\\]^_`abcdefghijklmnopqrstuvwxyz{|}~\x7f"
        self.compile_and_run(f'''
            let s1 = "{all_chars}";
            let *10 = s1;
        ''')
        addr = self.cpu.ram[10]
        for i, c in enumerate(all_chars):
            self.assertEqual(self.cpu.ram[addr+i], ord(c))

    def test_array(self):
        self.compile_and_run('''
            let a = 20;
            let arr = [1, 3+2, a+5, (8-3)+a, 9];
            let *10 = arr;
            let *11 = [1, 2 ,3];
        ''')
        addr = self.cpu.ram[10]
        self.assertEqual(self.cpu.ram[addr+0], 1)
        self.assertEqual(self.cpu.ram[addr+1], 3+2)
        self.assertEqual(self.cpu.ram[addr+2], 20+5)
        self.assertEqual(self.cpu.ram[addr+3], (8-3)+20)
        self.assertEqual(self.cpu.ram[addr+4], 9)
        addr = self.cpu.ram[11]
        self.assertEqual(self.cpu.ram[addr+0], 1)
        self.assertEqual(self.cpu.ram[addr+1], 2)
        self.assertEqual(self.cpu.ram[addr+2], 3)
        self.assertEqual(self.cpu.ram[0], 0x3fff)

    def test_nested_array(self):
        self.compile_and_run('''
            let *10 = [
                [1, 2, 3],
                [4, 5, 6]
            ];
        ''')
        outer = self.cpu.ram[10]
        inner1 = self.cpu.ram[outer+0]
        inner2 = self.cpu.ram[outer+1]
        self.assertEqual(self.cpu.ram[inner1+0], 1)
        self.assertEqual(self.cpu.ram[inner1+1], 2)
        self.assertEqual(self.cpu.ram[inner1+2], 3)
        self.assertEqual(self.cpu.ram[inner2+0], 4)
        self.assertEqual(self.cpu.ram[inner2+1], 5)
        self.assertEqual(self.cpu.ram[inner2+2], 6)

    def test_array_deref_arithmetic(self):
        self.compile_and_run('''
            let data = [1,2,3,4,5];
            let *10 = *data;
            let *11 = *(data+1);
            let *12 = *(data+2);
            let *13 = *(data+4);
        ''')
        self.assertEqual(self.cpu.ram[10], 1)
        self.assertEqual(self.cpu.ram[11], 2)
        self.assertEqual(self.cpu.ram[12], 3)
        self.assertEqual(self.cpu.ram[13], 5)

    def test_nested_array_deref_arithmetic(self):
        self.compile_and_run('''
            let table = [
                [0,1,2,3,4],
                [5,6,7,8,9]
            ];
            let *10 = *((*table)+3);  // row 0, col 3
            let *11 = *((*(table+1))+2);  // row 1, col 2

        ''')
        self.assertEqual(self.cpu.ram[10], 3)
        self.assertEqual(self.cpu.ram[11], 7)

run_test(TestJoy20)

.........................................
----------------------------------------------------------------------
Ran 41 tests in 4.005s

OK


# ตัวอย่างโค้ดส่งท้าย
ถึงจุดนี้ เราสามารถสร้างโค้ดแสดงผลบิตแมปบนหน้าจออย่างง่ายได้ด้วย Joy 2.0 ดังตัวอย่าง

In [ ]:
asm = Joy20().compile('''
    let smile = [
        0b0000011110000000,
        0b0001100001100000,
        0b0010000000010000,
        0b0100000000001000,
        0b0100100001001000,
        0b1001010010100100,
        0b1000000000000100,
        0b1000000000000100,
        0b1000100001000100,
        0b0100100001001000,
        0b0100011110001000,
        0b0010000000010000,
        0b0001100001100000,
        0b0000011110000000,
        0b0000000000000000,
        0b0000000000000000 ];

    let start = 2500;  // start word (from 0x4000) to draw on screen
    let length = 20;   // how many times to draw
    while length > 0 {
        let count = 0;
        let row = 0;
        while count < 16 { // draw each bitmap row
            let *(0x4000 + start + row) = *(smile+count);
            let count = count + 1;
            let row = row + 32;  // set row to the next line
        }
        let start = start + 1;    // set start to the next word
        let length = length - 1;
    }
    while 1 {} // program stops here
''')
assemble_and_simulate(asm, super_fast=True)